In [43]:

#    Multi-Disciplinary Design Optimisation of a Hydrogen Powered Commercial Aircraft (MDOHPCA)
#    this is the master file that runs each of the individual disciplines 
#    It sets up each of the analyses, then converts it into an openMDAO problem with a certain structure, then solves for minimum mass



In [44]:
import openmdao.api as om
import pandas as pd
import numpy as np

In [45]:
class MDOHPCA(om.Group):
    """
    Top level group containing the MDO.
    """           
    def setup(self):
        """
        set up each person's disciplines that can contain analyses
        """
        self.add_subsystem('aero', aero())
        self.add_subsystem('sys', sys())
        self.add_subsystem('stab', stab())
        self.add_subsystem('struct', struct())
        
        self.add_subsystem('mass', mass())
        
    def configure(self):
        #promote all variables (lazy option, they can be connected individually)
        self.promotes('sys',any=['*'])
        self.promotes('struct',any=['*'])
        
        self.promotes('mass',any=['*'])

In [46]:
class mass(om.ExplicitComponent):
    def setup(self):
        self.add_input('systems_mass')
        self.add_input('weight_structures')
        
        self.add_output('m')
        self.add_objective('m')
        
    def compute(self,inputs,outputs):
        outputs['m'] = inputs['systems_mass'] + inputs['weight_structures']

In [47]:
%run struct/structsetup.ipynb
#%run stab/stabsetup.ipynb
%run systems/systemssetup.ipynb
class aero(om.ExplicitComponent):
    pass
class stab(om.ExplicitComponent):
    pass

In [48]:
prob = om.Problem(model = MDOHPCA())
#prob.model.add_subsystem('aero', om.Group())

In [49]:
#setting inputs to reduce ambiguities where inputs differ in different disciplines
prob.model.set_input_defaults('CL', val = 0.6)
prob.model.set_input_defaults('sweep', val = 15)
prob.model.set_input_defaults('c', val = 4)
prob.model.set_input_defaults('b', val = 60)
prob.model.set_input_defaults('root_x', val = 10)
prob.model.set_input_defaults('m', val = 240000)

prob.model.add_design_var('fuselage_diameter')

In [50]:
#declare problem options (driver, optimiser)
prob.model.driver = om.ScipyOptimizeDriver()
prob.model.driver.options['optimizer'] = 'SLSQP'
#prob.nonlinear_solver = om.NonlinearBlockGS()
#prob.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
prob.model.nonlinear_solver = om.NonlinearBlockGS()
#prob.nonlinear_solver.options['iprint'] = 2
#prob.nonlinear_solver.options['maxiter'] = 100
prob.model.driver.options['maxiter'] = 5
prob.model.driver.options['tol'] = 1e-8

#setup problem to be run
prob.setup()

In [51]:
#%run aero/aerosetup.ipynb

In [52]:
prob.run_model()

NL: NLBGS Converged in 3 iterations


In [30]:
#get properties of specific constants in disciplines
prob.model.sys.options._dict['pp_mass']

{'val': 32816.0,
 'values': None,
 'types': None,
 'desc': 'people and cargo mass',
 'upper': None,
 'lower': None,
 'check_valid': None,
 'has_been_set': True,
 'allow_none': False,
 'recordable': True,
 'deprecation': None}

In [53]:
prob.model.list_outputs()

27 Explicit Output(s) in 'model'

varname                         val                 
------------------------------  --------------------
sys
  fuel_mass_calc
    fuel_mass                   [63.42212651]       
  tanks
    tank1_mass_full             [1429.67182543]     
    tank1_mass_empty            [1391.61854952]     
    tank2_mass_full             [1414.79580524]     
    tank2_mass_empty            [1389.42695464]     
    CGtp                        [30.50658828]       
    tank1_x                     [4.80864777]        
    tank2_x                     [46.42383765]       
  pipes
    pipe_CG                     [22.44224705]       
    pipe_mass                   [74.79668214]       
  engine
    engine_mass                 [1054.575]          
    engine_x                    [14.82308546]       
    engine_y                    [18.]               
  actuator
    each_flap_actuator_mass     [2.55]              
    each_aileron_actuator_mass  [0.1275]            
    actu

[('sys.fuel_mass_calc.fuel_mass', {'val': array([63.42212651])}),
 ('sys.tanks.tank1_mass_full', {'val': array([1429.67182543])}),
 ('sys.tanks.tank1_mass_empty', {'val': array([1391.61854952])}),
 ('sys.tanks.tank2_mass_full', {'val': array([1414.79580524])}),
 ('sys.tanks.tank2_mass_empty', {'val': array([1389.42695464])}),
 ('sys.tanks.CGtp', {'val': array([30.50658828])}),
 ('sys.tanks.tank1_x', {'val': array([4.80864777])}),
 ('sys.tanks.tank2_x', {'val': array([46.42383765])}),
 ('sys.pipes.pipe_CG', {'val': array([22.44224705])}),
 ('sys.pipes.pipe_mass', {'val': array([74.79668214])}),
 ('sys.engine.engine_mass', {'val': array([1054.575])}),
 ('sys.engine.engine_x', {'val': array([14.82308546])}),
 ('sys.engine.engine_y', {'val': array([18.])}),
 ('sys.actuator.each_flap_actuator_mass', {'val': array([2.55])}),
 ('sys.actuator.each_aileron_actuator_mass', {'val': array([0.1275])}),
 ('sys.actuator.actuators_CG_x', {'val': array([13.50885847])}),
 ('sys.systems_roundup.systems_m